In [1]:
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

# Prepare Data

In [2]:
data = pd.read_csv('../../../data/adult.data.csv')

## Drop categorical features
data = data.drop(['fnlwgt','education'],axis=1)

## Drop NULL values
data = data.dropna()

## for dowhy

data['Probability'] = np.where(data['Probability'] == " <=50K", 0, 1)
data['sex'] = np.where(data['sex'] == " Male", 1, 0)
data['race'] = np.where(data['race'] == " White", 1, 0)

non_numeric_columns = list(data.select_dtypes(exclude=[np.number]).columns)

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    data[col] = le.fit_transform(data[col])

print(non_numeric_columns)

from sklearn.preprocessing import MinMaxScaler,StandardScaler

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)
(data.head(5))

['workclass', 'marital_status', 'occupation', 'relationship', 'native_country']
        age  workclass  education_num  marital_status  occupation  \
0  0.301370      0.875       0.800000        0.666667    0.071429   
1  0.452055      0.750       0.800000        0.333333    0.285714   
2  0.287671      0.500       0.533333        0.000000    0.428571   
3  0.493151      0.500       0.400000        0.333333    0.428571   
4  0.150685      0.500       0.800000        0.333333    0.714286   

   relationship  race  sex  capital_gain  capital_loss  hours_per_week  \
0           0.2   1.0  1.0       0.02174           0.0        0.397959   
1           0.0   1.0  1.0       0.00000           0.0        0.122449   
2           0.2   1.0  1.0       0.00000           0.0        0.397959   
3           0.0   0.0  1.0       0.00000           0.0        0.397959   
4           1.0   0.0  0.0       0.00000           0.0        0.397959   

   native_country  Probability  
0        0.951220          

# Transforming data by LFR

In [3]:
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.datasets import BinaryLabelDataset
from IPython.display import Markdown, display

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

protected_attribute = "sex"
data1,dataset_orig_test = train_test_split(data,test_size=0.2,random_state=0)
X_train, y_train =  data1.loc[:, data1.columns != 'Probability'], data1['Probability']
X_test, y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

privileged_groups = [{protected_attribute: 1}]
unprivileged_groups = [{protected_attribute: 0}]

TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.1, Ay=1, Az=2,
         verbose=1)

dataset_orig1 = BinaryLabelDataset(df=data1, label_names=['Probability'], protected_attribute_names=[protected_attribute])

TR.fit(dataset_orig1, maxiter=5000, maxfun=5000)

dataset_transf = TR.transform(dataset_orig1)

step: 0, loss: 0.7291404741653125, L_x: 0.25964293440347247,  L_y: 0.6745982146110635,  L_z: 0.014288983056950842
step: 250, loss: 0.6528869267876272, L_x: 0.2587397795444081,  L_y: 0.6005785642331125,  L_z: 0.013217192300036942
step: 500, loss: 0.588370882872371, L_x: 0.2525687494779225,  L_y: 0.5455487596425248,  L_z: 0.008782624141026995
step: 750, loss: 0.6065248934343175, L_x: 0.20824948321694464,  L_y: 0.5464217996757036,  L_z: 0.019639072718459712
step: 1000, loss: 0.5797997782542352, L_x: 0.2405839078890577,  L_y: 0.5440375991511441,  L_z: 0.005851894157092618
step: 1250, loss: 0.5776248895968202, L_x: 0.23846188650388578,  L_y: 0.543134023573522,  L_z: 0.005322338686454818
step: 1500, loss: 0.5702710388352253, L_x: 0.22135270189051476,  L_y: 0.5396834778283065,  L_z: 0.004226145408933676
step: 1750, loss: 0.5658970710897983, L_x: 0.20030924177837447,  L_y: 0.5368368902499002,  L_z: 0.0045146283310302905
step: 2000, loss: 0.5625091086661053, L_x: 0.20280647996985757,  L_y: 0.53

In [ ]:
from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy

df, yy = dataset_transf.convert_to_dataframe()
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100) # LSR
X_train, y_train = df.loc[:, df.columns != 'Probability'], df['Probability']
clf.fit(X_train,y_train)
# print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'recall'))
# print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'far'))
# print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'precision'))
# print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'accuracy'))
# print("F1 Score :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'F1'))
print("aod :"+protected_attribute,measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'aod'))
print("eod :"+protected_attribute,measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'eod'))

print("SPD:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'SPD'))
print("DI:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, protected_attribute, 'DI'))

# Graph

In [ ]:
causal_graph = """
digraph {
marital_status;
workclass;
occupation;
relationship;
race;
sex;
education_num;
Probability;
U[label="Unobserved Confounders"];
sex -> Probability;
sex -> marital_status; marital_status -> Probability; workclass -> occupation;
marital_status -> occupation; occupation -> Probability
marital_status -> relationship; relationship -> Probability
race -> occupation; race -> Probability;
race -> education_num; education_num -> workclass;
workclass -> Probability;
U->sex;U->race;U->Probability;
}
"""

## dowhy works for binary inputs only

df['Probability'] = np.where(df['Probability'] == 0, False, True)
df[protected_attribute] = np.where(df[protected_attribute] == 1, True, False)

In [ ]:
# With graph
model=CausalModel(
        data = df,
        treatment=protected_attribute,
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )
model.view_model()

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)